<a href="https://colab.research.google.com/github/graviraja/100-Days-of-NLP/blob/applications%2Fquestion-answering/applications/question-answering/Basic%20Question%20Answering%20with%20Dynamic%20Memory%20Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### bAbI dataset

In [1]:
!wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz

--2020-07-30 18:26:15--  http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz
Resolving www.thespermwhale.com (www.thespermwhale.com)... 69.65.3.213
Connecting to www.thespermwhale.com (www.thespermwhale.com)|69.65.3.213|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15719851 (15M) [application/x-gzip]
Saving to: ‘tasks_1-20_v1-2.tar.gz’

tasks_1-20_v1-2.tar 100%[===================>]  14.99M  8.15MB/s    in 1.8s    

2020-07-30 18:26:17 (8.15 MB/s) - ‘tasks_1-20_v1-2.tar.gz’ saved [15719851/15719851]



In [2]:
!tar -xzf tasks_1-20_v1-2.tar.gz

In [3]:
!ls tasks_1-20_v1-2/en-10k/

qa10_indefinite-knowledge_test.txt   qa1_single-supporting-fact_test.txt
qa10_indefinite-knowledge_train.txt  qa1_single-supporting-fact_train.txt
qa11_basic-coreference_test.txt      qa20_agents-motivations_test.txt
qa11_basic-coreference_train.txt     qa20_agents-motivations_train.txt
qa12_conjunction_test.txt	     qa2_two-supporting-facts_test.txt
qa12_conjunction_train.txt	     qa2_two-supporting-facts_train.txt
qa13_compound-coreference_test.txt   qa3_three-supporting-facts_test.txt
qa13_compound-coreference_train.txt  qa3_three-supporting-facts_train.txt
qa14_time-reasoning_test.txt	     qa4_two-arg-relations_test.txt
qa14_time-reasoning_train.txt	     qa4_two-arg-relations_train.txt
qa15_basic-deduction_test.txt	     qa5_three-arg-relations_test.txt
qa15_basic-deduction_train.txt	     qa5_three-arg-relations_train.txt
qa16_basic-induction_test.txt	     qa6_yes-no-questions_test.txt
qa16_basic-induction_train.txt	     qa6_yes-no-questions_train.txt
qa17_positional-reasoning_test.

In [4]:
train_file = 'tasks_1-20_v1-2/en-10k/qa5_three-arg-relations_train.txt'
test_file = 'tasks_1-20_v1-2/en-10k/qa5_three-arg-relations_test.txt'

### Imports

In [5]:
import os
import re

import time
import nltk
import random
import pickle
import unicodedata
import numpy as np

from collections import Counter
from copy import deepcopy

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
from torch.autograd import Variable

from sklearn.model_selection import train_test_split

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Data Exploration

In [7]:
with open(train_file) as f:
    train_file_data = f.readlines()

In [8]:
train_file_data = [d[:-1] for d in train_file_data]

In [9]:
train_file_data[:30]

['1 Bill travelled to the office.',
 '2 Bill picked up the football there.',
 '3 Bill went to the bedroom.',
 '4 Bill gave the football to Fred.',
 '5 What did Bill give to Fred? \tfootball\t4',
 '6 Fred handed the football to Bill.',
 '7 Jeff went back to the office.',
 '8 Who received the football? \tBill\t6',
 '9 Bill travelled to the office.',
 '10 Bill got the milk there.',
 '11 Who received the football? \tBill\t6',
 '12 Fred travelled to the garden.',
 '13 Fred went to the hallway.',
 '14 Bill journeyed to the bedroom.',
 '15 Jeff moved to the hallway.',
 '16 Jeff journeyed to the bathroom.',
 '17 Bill journeyed to the office.',
 '18 Fred travelled to the bathroom.',
 '19 Mary journeyed to the kitchen.',
 '20 Jeff took the apple there.',
 '21 Jeff gave the apple to Fred.',
 '22 Who did Jeff give the apple to? \tFred\t21',
 '23 Bill went back to the bathroom.',
 '24 Bill left the milk.',
 '25 Who received the apple? \tFred\t21',
 '1 Mary travelled to the garden.',
 '2 Mary journe

In [10]:
def load_data(data):
    qa_data = []
    fact = []
    qa = []

    for d in data:
        index = d.split()[0]
        if index == "1":
            fact = []
            qa = []
        if '?' in d:
            temp = d.split('\t')
            q = temp[0].strip().replace('?', '').split()[1:] + ['?']
            a = temp[1].split() + ['</s>']
            stemp = deepcopy(fact)
            qa_data.append([stemp, q, a])
        else:
            tokens = d.replace('.', '').split()[1:] + ['</s>']
            fact.append(tokens)
    return qa_data

In [30]:
train_data = load_data(train_file_data)

In [31]:
len(train_data)

10000

In [32]:
train_data[0], train_data[1]

([[['Bill', 'travelled', 'to', 'the', 'office', '</s>'],
   ['Bill', 'picked', 'up', 'the', 'football', 'there', '</s>'],
   ['Bill', 'went', 'to', 'the', 'bedroom', '</s>'],
   ['Bill', 'gave', 'the', 'football', 'to', 'Fred', '</s>']],
  ['What', 'did', 'Bill', 'give', 'to', 'Fred', '?'],
  ['football', '</s>']],
 [[['Bill', 'travelled', 'to', 'the', 'office', '</s>'],
   ['Bill', 'picked', 'up', 'the', 'football', 'there', '</s>'],
   ['Bill', 'went', 'to', 'the', 'bedroom', '</s>'],
   ['Bill', 'gave', 'the', 'football', 'to', 'Fred', '</s>'],
   ['Fred', 'handed', 'the', 'football', 'to', 'Bill', '</s>'],
   ['Jeff', 'went', 'back', 'to', 'the', 'office', '</s>']],
  ['Who', 'received', 'the', 'football', '?'],
  ['Bill', '</s>']])

In [33]:
len(train_data[0])

3

In [34]:
train_data, eval_data = train_test_split(train_data, test_size=0.2, random_state=42)
len(train_data), len(eval_data)

(8000, 2000)

### Vocabulary

In [35]:
PAD_TOKEN = 0
START_TOKEN = 1
END_TOKEN = 2
UNK_TOKEN = 3

In [36]:
class Vocab(object):
    def __init__(self):
        self.word2id = {
            "<s>": START_TOKEN,
            "</s>": END_TOKEN,
            "<pad>": PAD_TOKEN,
            "<unk>": UNK_TOKEN
        }

        self.id2word = dict((id, word) for word, id in self.word2id.items())
        self.length = 4
    
    def add_word(self, word):
        if word not in self.word2id:
            self.word2id[word] = self.length
            self.id2word[self.length] = word
            self.length += 1
    
    def __len__(self):
        return self.length
    
    def __call__(self, word):
        if not word in self.word2id:
            return self.word2id['<unk>']
        return self.word2id[word]

In [37]:
def build_vocab(data, min_count=5):
    facts, questions, answers = list(zip(*data))
    vocab = Vocab()
    counter = Counter()

    for fact in facts:
        for f in fact:
            counter.update(f)
    
    for q in questions:
        counter.update(q)
    
    for a in answers:
        counter.update(a)
    
    for word, count in counter.most_common():
        if count >= min_count:
            vocab.add_word(word)
    
    vocab_file = 'vocab.pkl'
    print("saving vocabulary...")
    with open(vocab_file, 'wb') as f:
        pickle.dump(vocab, f)

In [38]:
def load_vocab():
    with open('vocab.pkl', 'rb') as f:
        vocab = pickle.load(f)
    return vocab

In [39]:
build_vocab(train_data)

saving vocabulary...


In [40]:
vocab = load_vocab()

In [41]:
len(vocab)

44

In [42]:
list(vocab.word2id.keys())

['<s>',
 '</s>',
 '<pad>',
 '<unk>',
 'the',
 'to',
 'Mary',
 'Fred',
 'Bill',
 'Jeff',
 'went',
 'there',
 'apple',
 'football',
 'milk',
 'moved',
 'travelled',
 'journeyed',
 'back',
 'garden',
 'office',
 'hallway',
 'kitchen',
 'bathroom',
 'bedroom',
 'gave',
 '?',
 'took',
 'passed',
 'grabbed',
 'got',
 'picked',
 'up',
 'handed',
 'Who',
 'left',
 'dropped',
 'put',
 'down',
 'discarded',
 'did',
 'give',
 'What',
 'received']

### Dataset

In [57]:
def get_batch(data, batch_size=64):
    start_idx = 0
    end_idx = batch_size
    while end_idx < len(data):
        batch = data[start_idx: end_idx]
        start_idx = end_idx
        end_idx = end_idx + batch_size
        yield batch
    
    if end_idx >= len(data):
        batch = data[start_idx:]
        yield batch

In [44]:
def prepare_seq(seq):
    idxs = [vocab(tok) for tok in seq]
    return torch.LongTensor(idxs)

In [45]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [67]:
def pad_batch(batch):
    fact, ques, ans = list(zip(*batch))

    max_facts = max([len(f) for f in fact])
    max_fact_len = max([f.size(1) for f in flatten(fact)])
    max_ques_len = max([q.size(1) for q in ques])
    max_ans_len = max([a.size(1) for a in ans])

    facts, questions, answers = [], [], []

    for i in range(len(batch)):
        fact_t = []
        fact_l = []
        for j in range(len(fact[i])):
            if fact[i][j].size(1) < max_ques_len:
                fact_l.append(fact[i][j].size(1))
                fact_t.append(torch.cat([fact[i][j], Variable(torch.LongTensor([[vocab('<pad>')] * (max_fact_len - fact[i][j].size(1))])).view(1, -1)], 1))
            else:
                fact_t.append(fact[i][j])
                fact_l.append(fact[i][j].size(1))
        
        while len(fact_t) < max_facts:
            fact_t.append(Variable(torch.LongTensor([vocab('<pad>')] * max_fact_len)).view(1, -1))

        fact_t = torch.cat(fact_t)
        facts.append(fact_t.unsqueeze(0))

        # padding question
        if ques[i].size(1) < max_ques_len:
            questions.append(torch.cat([ques[i], Variable(torch.LongTensor([[vocab('<pad>')] * (max_ques_len - ques[i].size(1))])).view(1, -1)], 1))
        else:
            questions.append(ques[i])
        
        # padding answer
        if ans[i].size(1) < max_ans_len:
            answers.append(torch.cat([ans[i], Variable(torch.LongTensor([[vocab('<pad>')] * (max_ans_len - ans[i].size(1))])).view(1, -1)], 1))
        else:
            answers.append(ans[i])

    facts = torch.cat(facts).to(device)
    fact_masks = facts == 0
    fact_masks.to(device)
    questions = torch.cat(questions).to(device)
    answers = torch.cat(answers).to(device)
    question_masks = questions == 0
    question_masks = question_masks.to(device)
    
    return facts, fact_masks, questions, question_masks, answers


In [47]:
for t in train_data:
    for i, fact in enumerate(t[0]):
        t[0][i] = prepare_seq(fact).view(1, -1)
    t[1] = prepare_seq(t[1]).view(1, -1)
    t[2] = prepare_seq(t[2]).view(1, -1)

In [48]:
for t in eval_data:
    for i, fact in enumerate(t[0]):
        t[0][i] = prepare_seq(fact).view(1, -1)
    t[1] = prepare_seq(t[1]).view(1, -1)
    t[2] = prepare_seq(t[2]).view(1, -1)

In [59]:
sample_batch = get_batch(train_data)

### Model

In [83]:
class DMN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.2):
        super().__init__()

        self.hidden_size = hidden_size
        self.embed = nn.Embedding(input_size, hidden_size, padding_idx=0)
        self.input_gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.question_gru = nn.GRU(hidden_size, hidden_size, batch_first=True)

        self.gate = nn.Sequential(
            nn.Linear(hidden_size * 4, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()
        )

        self.attention_grucell = nn.GRUCell(hidden_size, hidden_size)
        self.memory_grucell = nn.GRUCell(hidden_size, hidden_size)
        self.answer_grucell = nn.GRUCell(hidden_size * 2, hidden_size)
        self.answer_fc = nn.Linear(hidden_size, output_size)

        self.dropout = nn.Dropout(dropout)
    
    def init_hidden(self, inputs):
        # inputs => [num_inp, max_inp_size]
        hidden = Variable(torch.zeros(1, inputs.size(0), self.hidden_size)).to(device)
        # hidden => [1, num_inp, hid_dim]

        return hidden
    
    def forward(self, facts, facts_masks, questions, question_masks, num_decode, episodes=3, is_training=False):
        # facts => [batch_size, num_facts, max_fact_length]
        # questions => [batch_size, max_ques_length]

        # Input Module
        fact_reps = []
        for fact, fact_mask in zip(facts, facts_masks):
            f_embeds = self.embed(fact)
            f_embeds = self.dropout(f_embeds)
            # f_embeds => [num_facts, max_fact_length, hid_dim]
            
            hidden = self.init_hidden(fact)
            # hidden => [1, num_facts, hid_dim]

            outputs, hidden = self.input_gru(f_embeds, hidden)
            # outputs => [num_facts, max_fact_length, hid_dim]
            
            fact_rep = []
            for i, o in enumerate(outputs):
                fact_length = fact_mask[i].data.tolist().count(0)
                # get the final hidden state of the fact
                # since the facts are padded, get the original length from mask
                fact_rep.append(o[fact_length - 1])     # neglect the </s> input
            
            # fact_rep => [num_facts], each => [hid_dim]
            fact_rep = torch.cat(fact_rep)
            # fact_rep => [num_facts, hid_dim]
            
            # add extra dimension, so that it can used to merge later on for the batch
            fact_rep = fact_rep.view(fact.size(0), -1).unsqueeze(0)
            # fact_rep => [1, num_facts, hid_dim]

            fact_reps.append(fact_rep)
        
        encoded_facts = torch.cat(fact_reps)
        # encoded_facts => [batch_size, num_facts, hid_dim]

        # Question Module
        q_embeds = self.embed(questions)
        q_embeds = self.dropout(q_embeds)
        # q_embeds => [batch_size, max_ques_len, hid_dim]

        hidden = self.init_hidden(questions)
        q_outputs, hidden = self.question_gru(q_embeds, hidden)
        # q_outputs => [batch_size, max_ques_len, hid_dim]
        # hidden => [1, batch_size, hid_dim]

        if is_training:
            q_rep = []
            for i, o in enumerate(q_outputs):
                q_length = question_masks[i].data.tolist().count(0)
                q_rep.append(o[q_length - 1])   # neglect the </s> input
            q_rep = torch.cat(q_rep)
            # q_rep => [batch_size, hid_dim]

            encoded_question = q_rep.view(questions.size(0), -1)
            # encoded_question => [batch_size, hid_dim]
        else:
            encoded_question = hidden.squeeze(0)
            # encoded_question => [batch_size, hid_dim]

        # Episodic Memory Module
        memory = encoded_question
        # encoded_facts => [batch_size, num_facts, hid_dim]
        num_facts = encoded_facts.size(1)
        batch_size = encoded_facts.size(0)

        for t in range(episodes):
            hidden = self.init_hidden(encoded_facts.transpose(0, 1)[0]).squeeze(0)
            # hidden => [batch_size, hid_dim]

            for each_fact in range(num_facts):
                z = torch.cat(
                    [
                        encoded_facts.transpose(0, 1)[t] * encoded_question,            # element wise product => [batch_size, hid_dim]
                        encoded_facts.transpose(0, 1)[t] * memory,                      # element wise product => [batch_size, hid_dim]
                        torch.abs(encoded_facts.transpose(0, 1)[t] - encoded_question), # element wise diff => [batch_size, hid_dim]
                        torch.abs(encoded_facts.transpose(0, 1)[t] - memory)            # element wise diff => [batch_size, hid_dim]
                    ], dim=1
                )
                # z => [batch_size, hid_dim * 4]

                g_t = self.gate(z)
                # g_t => [batch_size, 1]
                hidden = g_t * self.attention_grucell(encoded_facts.transpose(0, 1)[t], hidden) + (1 - g_t) * hidden
            
            e = hidden
            memory = self.memory_grucell(e, memory)


        # Answer Module
        answer_hidden = memory
        start_decode = Variable(torch.LongTensor([[vocab('<s>')] * memory.size(0)])).transpose(0, 1)
        start_decode = start_decode.to(device)
        y_t_1 = self.embed(start_decode).squeeze(1)
        # y_t_1 => [batch_size, hid_dim]

        decodes = []
        for t in range(num_decode):
            answer_hidden = self.answer_grucell(torch.cat([y_t_1, encoded_question], 1), answer_hidden)
            decodes.append(F.log_softmax(self.answer_fc(answer_hidden), 1))
        return torch.cat(decodes, 1).view(batch_size * num_decode, -1)


In [90]:
HIDDEN_SIZE = 80
NUM_EPISODE = 3

model = DMN(len(vocab), HIDDEN_SIZE, len(vocab)).to(device)

### Optimizer & Criterion

In [91]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [92]:
def train(model, dataset):
    model.train()

    epoch_loss = 0
    total = 0
    for i, batch in enumerate(get_batch(dataset)):
        facts, fact_masks, questions, question_masks, answers = pad_batch(batch)

        model.zero_grad()
        predictions = model(facts, fact_masks, questions, question_masks, answers.size(1), NUM_EPISODE, True)
        loss = criterion(predictions, answers.view(-1))
        
        epoch_loss += loss.item()
        total += 1

        loss.backward()
        optimizer.step()
    
    return epoch_loss / total

In [93]:
def evaluate(model, dataset):
    model.eval()

    epoch_loss = 0
    total = 0
    with torch.no_grad():
        for i, batch in enumerate(get_batch(dataset)):
            facts, fact_masks, questions, question_masks, answers = pad_batch(batch)

            predictions = model(facts, fact_masks, questions, question_masks, answers.size(1), NUM_EPISODE, False)
            loss = criterion(predictions, answers.view(-1))
            
            epoch_loss += loss.item()
            total += 1
    
    return epoch_loss / total

In [94]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [95]:
n_epochs = 10

best_loss = float('inf')
no_improvement = 0
patience = 3

for epoch in range(n_epochs):
    start_time = time.time()
    train_loss = train(model, train_data)
    eval_loss = evaluate(model, eval_data)
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Val. Loss: {eval_loss:.3f}')

    if eval_loss < best_loss:
        best_loss = eval_loss
        no_improvement = 0
        torch.save(model.state_dict(), 'model.pt')
    else:
        no_improvement += 1
    
    if no_improvement == patience:
        break

Epoch: 01 | Time: 1m 43s
	Train Loss: 1.168 | Val. Loss: 0.679
Epoch: 02 | Time: 1m 43s
	Train Loss: 0.660 | Val. Loss: 0.653
Epoch: 03 | Time: 1m 44s
	Train Loss: 0.645 | Val. Loss: 0.641
Epoch: 04 | Time: 1m 44s
	Train Loss: 0.628 | Val. Loss: 0.621
Epoch: 05 | Time: 1m 44s
	Train Loss: 0.612 | Val. Loss: 0.612
Epoch: 06 | Time: 1m 44s
	Train Loss: 0.605 | Val. Loss: 0.607
Epoch: 07 | Time: 1m 42s
	Train Loss: 0.602 | Val. Loss: 0.606
Epoch: 08 | Time: 1m 42s
	Train Loss: 0.600 | Val. Loss: 0.605
Epoch: 09 | Time: 1m 42s
	Train Loss: 0.599 | Val. Loss: 0.604
Epoch: 10 | Time: 1m 43s
	Train Loss: 0.598 | Val. Loss: 0.603
